<a href="https://colab.research.google.com/github/Gaurav7004/Textual-Analysis-ILH/blob/main/Data_Extraction_from_QPRs_17thSept2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install nltk
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip install gensim
# !pip install sumy
# !pip install python-docx
# !pip install pdfplumber

In [ ]:
# import re
# import pdfplumber

# def extract_text_from_pdf(pdf_path):
#     text = ''
#     with pdfplumber.open(pdf_path) as pdf:
#         for page in pdf.pages:
#             # Extract text from each page
#             page_text = page.extract_text()
#             if page_text:
#                 text += page_text + '\n'
#     return text

# def find_section_boundaries(text):
#     # Define regex patterns for different types of section headings
#     section_patterns = [
#         r'^\s*([IVX]+)\.\s+.*$',                   # Matches top-level sections like "I. INTRODUCTION"
#         r'^\s*RESULT\s+\d+:.*$',                   # Matches sections like "RESULT 1: ..."
#         r'^\s*ANNEX\s+\d+:.*$',                    # Matches annex sections like "ANNEX 1: ..."
#         r'^\s*[A-Z][A-Z\s/&\-\(\)]{2,}$',          # Matches headings in all caps (with spaces and certain symbols)
#         r'^\s*BENEFICIARY FEEDBACK.*$',            # Matches specific headings like "BENEFICIARY FEEDBACK"
#     ]

#     # Combine all patterns into one for efficiency
#     combined_pattern = re.compile('|'.join(section_patterns), re.MULTILINE)

#     # Find all matches in the text
#     matches = list(combined_pattern.finditer(text))

#     # Build a list of sections with their start and end indices
#     sections = []
#     for i, match in enumerate(matches):
#         start = match.start()
#         title = match.group().strip()
#         if i + 1 < len(matches):
#             end = matches[i + 1].start()
#         else:
#             end = len(text)
#         content = text[start:end].strip()
#         sections.append({
#             'title': title,
#             'start': start,
#             'end': end,
#             'content': content
#         })

#     return sections

# # Example usage:
# if __name__ == "__main__":
#     # Path to your PDF file
#     pdf_path = 'QPR Sattva CLM-Jan-March 2024.pdf'  # Replace with the path to your PDF file

#     # Extract text from the PDF
#     extracted_text = extract_text_from_pdf(pdf_path)

#     # Find the position where the table of contents ends
#     # We can look for the first occurrence of "I. INTRODUCTION" or another main section heading
#     main_content_start = re.search(r'\nI\.\s+INTRODUCTION|\nI\.\s+SUMMARY', extracted_text)
#     if main_content_start:
#         # Trim the text to start from the main content
#         extracted_text = extracted_text[main_content_start.start():]
#     else:
#         print("Could not find the start of the main content.")
#         # Optionally, you can decide to proceed without trimming or handle this case differently

#     # Find sections
#     sections = find_section_boundaries(extracted_text)

#     # Define the desired section titles or keywords
#     desired_sections = ['SUMMARY', 'INTRODUCTION', 'HIGHLIGHTS OF THE QUARTER', 'MAJOR ACHIEVEMENTS']

#     # Filter the sections to include only the desired ones
#     selected_sections = []
#     for section in sections:
#         # Check if any of the desired keywords are in the section title
#         for phrase in desired_sections:
#             if phrase in section['title'].upper():
#                 selected_sections.append(section)
#                 break  # Stop checking other phrases if a match is found

#     # Print out the selected sections
#     for section in selected_sections:
#         print(f"Section: {section['title']}")
#         print(f"Content:\n{section['content']}\n")
#         print("=" * 100)


In [ ]:
import re
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + '\n'
    return text

def find_section_boundaries(text):
    # Replace multiple spaces and tabs with a single space, but keep newlines
    text = re.sub(r'[ \t]+', ' ', text)
    # Split text into lines
    lines = text.split('\n')
    sections = []
    current_section = None

    # Updated regex pattern to include 'RESULT' sections
    section_heading_regex = r'''
        ^\s*                    # Start of the line, optional whitespace
        (                       # Start capturing group
            [IVXLCDM]+\.        # Roman numerals followed by a dot
            |                   # OR
            RESULT\s*\d+:       # 'RESULT' followed by number and colon
        )
        .*                      # Any characters after the heading
        $
    '''
    pattern = re.compile(section_heading_regex, re.IGNORECASE | re.VERBOSE)

    for i, line in enumerate(lines):
        # Normalize whitespace in the line
        line = re.sub(r'\s+', ' ', line).strip()
        # Check if the line matches the section heading
        if pattern.match(line):
            if current_section:
                current_section['end'] = i
                sections.append(current_section)
            title = line.strip()
            print(f"Found section title: {title}")
            current_section = {'title': title, 'start': i, 'end': None}
    if current_section:
        current_section['end'] = len(lines)
        sections.append(current_section)

    # Combine lines for each section
    for section in sections:
        content_lines = lines[section['start']:section['end']]
        section['content'] = '\n'.join(content_lines).strip()
    return sections

if __name__ == "__main__":
    pdf_paths = [
        "MCGL India-Yash QPR Apr-Jun'24 31.07.2024.pdf",
        "MCGL India-Yash QPR Jan-Mar'24 29.04.2024 2027.pdf",
        "MCGL India-Yash QPR Oct-Dec - 22.01 - Clean.pdf"
    ]

    for pdf_path in pdf_paths:
        extracted_text = extract_text_from_pdf(pdf_path)

        main_content_start = re.search(r'I\.\s+INTRODUCTION|I\.\s+SUMMARY', extracted_text, re.IGNORECASE)
        if main_content_start:
            extracted_text = extracted_text[main_content_start.start():]
        else:
            print(f"Could not find the start of the main content in {pdf_path}.")

        sections = find_section_boundaries(extracted_text)

        desired_sections = [
            'SUMMARY',
            'INTRODUCTION',
            'HIGHLIGHTS',
            'MAJOR ACHIEVEMENTS',
            'RESULT'
        ]
        selected_sections = []
        for section in sections:
            section_title_upper = section['title'].upper()
            for phrase in desired_sections:
                if phrase.upper() in section_title_upper:
                    selected_sections.append(section)
                    break  # Stop checking other phrases once a match is found

        print(f"\nSelected sections from {pdf_path}:\n")
        for section in selected_sections:
            # print(f"Section: {section['title']}")
            print(f"Content:\n{section['content']}\n")
            print("=" * 100)


Found section title: I. INTRODUCTION ................................................................................................... 7
Found section title: II. HIGHLIGHTS OF THE QUARTER ............................................................................. 9
Found section title: RESULT 1: INCREASED AGENCY OF YOUNG POPULATIONS TO DEMAND FOR AND
Found section title: RESULT 2: IMPROVED ACCESSIBILITY AND AFFORDABILITY OF QUALITY FP/RH CARE
Found section title: RESULT 3: STRENGTHENED YOUTH-FRIENDLY ECOSYSTEM BY INCREASED
Found section title: III. MAJOR PLAN (FOR THE NEXT QUARTER) ............................................................. 28
Found section title: RESULT 1: INCREASED AGENCY OF YOUNG POPULATIONS TO DEMAND AND
Found section title: RESULT 2: IMPROVED ACCESSIBILITY AND AFFORDABILITY OF QUALITY FP/RH CARE
Found section title: RESULT 3: STRENGTHENED YOUTH-FRIENDLY ECOSYSTEM BY INCREASED
Found section title: RESULT 4: STRENGTHENED HEALTH SYSTEMS TO MAINTAIN ESSENTIAL FP

In [ ]:

import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from transformers import pipeline
from docx import Document
import os

# Selected Keywords
selected_keywords = [
    'gender', 'women', 'girl', 'adolescent', 'gender-based violence', 'GBV',
    'regressive gender norms', 'gender stereotypes', 'female feticide', 'gender equity',
    'women’s rights', 'child early and forced marriage', 'CEFM', 'gender champion'
]

def extract_text_from_pdfs(pdf_paths):
    import pdfplumber
    combined_text = ''
    for pdf_path in pdf_paths:
        if not os.path.exists(pdf_path):
            print(f"File not found: {pdf_path}")
            continue
        print(f"Processing PDF: {pdf_path}")
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    combined_text += page_text + '\n'
    print(f"Total length of combined text: {len(combined_text)} characters")
    return combined_text

def extract_sentences_by_keywords(text, keywords):
    sentences = sent_tokenize(text)
    print(f"Total number of sentences: {len(sentences)}")
    keyword_pattern = '|'.join([re.escape(k) for k in keywords])
    pattern = re.compile(r'\b(' + keyword_pattern + r')\b', re.IGNORECASE)
    keyword_sentences = {k.lower(): [] for k in keywords}

    for sentence in sentences:
        matches = pattern.findall(sentence)
        for match in matches:
            keyword_sentences[match.lower()].append(sentence)
    for keyword in keywords:
        print(f"Keyword '{keyword}': {len(keyword_sentences[keyword.lower()])} sentences found")
    return keyword_sentences

def summarize_text(text, summarizer):
    max_chunk_size = 800  # Adjust as needed
    text_chunks = [text[i:i+max_chunk_size] for i in range(0, len(text), max_chunk_size)]
    summaries = []
    for idx, chunk in enumerate(text_chunks):
        print(f"Summarizing chunk {idx+1}/{len(text_chunks)}")
        try:
            summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)
            summaries.append(summary[0]['summary_text'])
        except Exception as e:
            print(f"An error occurred during summarization: {e}")
    return ' '.join(summaries)

def summarize_sentences_by_keywords(keyword_sentences, summarizer):
    keyword_summaries = {}
    for keyword, sentences in keyword_sentences.items():
        print(f"Summarizing for keyword '{keyword}' with {len(sentences)} sentences")
        if sentences:
            summaries = []
            batch_size = 10  # Adjust as needed
            for i in range(0, len(sentences), batch_size):
                batch_sentences = sentences[i:i+batch_size]
                combined_text = ' '.join(batch_sentences)
                summary = summarize_text(combined_text, summarizer)
                summaries.append(summary)
            keyword_summaries[keyword] = ' '.join(summaries)
        else:
            keyword_summaries[keyword] = 'No relevant text found.'
    return keyword_summaries

def save_summaries_to_docx(keyword_summaries, output_filename):
    document = Document()
    for keyword, summary in keyword_summaries.items():
        document.add_heading(f'Keyword: {keyword.capitalize()}', level=1)
        document.add_paragraph(summary)
    document.save(output_filename)

if __name__ == "__main__":
    pdf_paths = [
        "MCGL India-Yash QPR Apr-Jun'24 31.07.2024.pdf"
        "MCGL India-Yash QPR Jan-Mar'24 29.04.2024 2027.pdf",
        "MCGL India-Yash QPR Oct-Dec - 22.01 - Clean.pdf"
    ]

    # Ensure PDFs are accessible
    for pdf_path in pdf_paths:
        if not os.path.exists(pdf_path):
            print(f"Please upload the file '{pdf_path}' to the Colab environment.")
            # You may need to upload the files or adjust the paths

    # Step 1: Extract text from PDFs
    combined_text = extract_text_from_pdfs(pdf_paths)

    # Step 2: Extract sentences by keywords
    keyword_sentences = extract_sentences_by_keywords(combined_text, selected_keywords)

    # Initialize summarizer once with a smaller model and use GPU if available
    device = 0 if torch.cuda.is_available() else -1
    summarizer = pipeline('summarization', model='t5-small', tokenizer='t5-small', device=device)

    # Step 3: Summarize sentences by keywords
    keyword_summaries = summarize_sentences_by_keywords(keyword_sentences, summarizer)

    # Step 4: Save summaries to a .docx file
    save_summaries_to_docx(keyword_summaries, 'keyword_summaries.docx')

    print("Summaries have been saved to 'keyword_summaries.docx'.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your max_length is set to 300, but your input_length is only 236. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=118)
Your max_length is set to 300, but your input_length is only 217. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=108)
Your max_length is set to 300, but your input_length is only 210. Since this is a summarization task, where outputs shorter than the input are typic

Summaries have been saved to 'keyword_summaries.docx'.


In [ ]:
import pandas as pd
import numpy as np
import torch
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    BertConfig,
)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =====================================
# Step 1: Prepare the Dataset
# =====================================

# Your CARS dataset in JSON format
cars_dataset = [
  {
    "event": "USAID partners with India to build equitable and inclusive healthcare access and climate-resilient health services.",
    "cause": "Recognizing the need for a robust healthcare system serving India's diverse population and aligning with India's National Health Policy.",
    "action": "Fostering partnerships, leveraging technology and innovation, and focusing on equitable healthcare access.",
    "result": "Improved health outcomes, strengthened health systems, and equitable access to quality healthcare.",
    "significance": "Demonstrates scalable global development solutions and enhances global health security."
  },
  {
    "event": "Creation of the world's first health-focused blended financing facility by USAID and local partners.",
    "cause": "Need for innovative financing to support healthcare enterprises and scale healthcare innovations.",
    "action": "Establishing a financing facility offering grants and debt financing to social entrepreneurs.",
    "result": "Invested in over 90 innovations, benefiting more than 43 million people and leveraging over $270 million in investments.",
    "significance": "Accelerated health innovation scaling and established a replicable financing model globally."
  },
  {
    "event": "Development and nationwide scaling of an AI tool to scan media for disease emergence.",
    "cause": "Need for enhanced disease surveillance and early outbreak detection.",
    "action": "Developing an AI tool to scan media for signs of disease and integrating it into national platforms.",
    "result": "Scanned 65 million articles in 2023, helping prevent an estimated 230 outbreaks.",
    "significance": "Improved global health security and provided a model for other countries."
  },
  {
    "event": "Scaling of maternal and child health models in India and Eastern Africa.",
    "cause": "High rates of infant respiratory distress and maternal deaths due to postpartum hemorrhage.",
    "action": "Demonstrated use of CPAP machines for infants and introduced new drugs and devices to reduce maternal deaths.",
    "result": "Improved maternal and child health outcomes and scaled health models in multiple regions.",
    "significance": "Contributed to reducing mortality rates and provided scalable health solutions globally."
  },
  {
    "event": "Transition of USAID HIV programs to models of entrepreneurship and empowerment for the LGBTQI+ community.",
    "cause": "Need to empower the LGBTQI+ community beyond traditional healthcare models and reduce stigma.",
    "action": "Established comprehensive clinics, supported civil society organizations, and promoted LGBTQI+ social enterprises.",
    "result": "Empowered the community with safe healthcare access and opportunities for self-sufficiency.",
    "significance": "Advanced equality, inclusion, and influenced global approaches to LGBTQI+ healthcare."
  },
  {
    "event": "Partnership with the Government of India to develop climate-resilient healthcare services.",
    "cause": "Impact of climate change on health and the necessity for sustainable healthcare infrastructure.",
    "action": "Developed solar-powered facilities, smart water usage, waste management, and greener supply chains; created state action plans.",
    "result": "Enhanced sustainability and resilience of healthcare services across 13 states.",
    "significance": "Set a precedent for integrating climate resilience into healthcare, aiding global efforts against climate change."
  },
  {
    "event": "Development of online platforms and networks to support young people's health decision-making.",
    "cause": "Need to improve access to healthcare information and empower youth in health decisions.",
    "action": "Helped develop digital platforms providing health information and products to youth.",
    "result": "Enhanced access to health resources and improved decision-making among young people.",
    "significance": "Empowered youth, contributing to long-term positive health outcomes and demonstrating effective use of technology."
  },
  {
    "event": "Facilitation of health learning exchanges between India and other countries.",
    "cause": "Need to share best practices and collaborate on global health challenges.",
    "action": "Organized exchanges with governments of Bangladesh, Fiji, Indonesia, Nepal, Nigeria, the Philippines, and Sri Lanka.",
    "result": "Enhanced collaboration, adoption of successful health practices, and strengthened global health networks.",
    "significance": "Promoted international cooperation, contributing to improved health outcomes globally."
  },
  {
    "event": "Embedding inclusive development principles in USAID's work at every level.",
    "cause": "Importance of diversity, equity, and accessibility in development programs.",
    "action": "Worked with community-based organizations and marginalized groups; implemented inclusive monitoring and evaluation processes.",
    "result": "Increased demand for equitable healthcare services and improved inclusivity in interventions.",
    "significance": "Strengthened impact of initiatives and set standards for inclusive practices in international development."
  },
  {
    "event": "USAID's efforts to enhance global health security and strengthen local health systems.",
    "cause": "Threat of global health crises and the need for robust local health systems.",
    "action": "Strengthened disease surveillance, responded to emerging infections, mobilized resources, and promoted research.",
    "result": "Improved ability to manage health challenges and prevent potential outbreaks.",
    "significance": "Reduced risk of global health emergencies and contributed to safer, healthier communities worldwide."
  },
  {
    "event": "USAID's collaboration to support TB elimination and HIV targets in alignment with India's health goals.",
    "cause": "The prevalence of TB and HIV in India and the need to meet elimination targets.",
    "action": "Supporting programs focused on TB elimination and HIV prevention and treatment.",
    "result": "Progress toward meeting TB elimination and HIV targets in India.",
    "significance": "Contributed to improved public health and aligned with national and global health priorities."
  },
  {
    "event": "USAID's work to ensure local voices guide the formulation of equitable and effective responses.",
    "cause": "Need for community engagement to enhance the effectiveness of health programs.",
    "action": "Collaborated with local communities, marginalized populations, and tribal groups to drive demand for comprehensive healthcare.",
    "result": "Programs that are more responsive to local needs and increased utilization of health services.",
    "significance": "Enhanced community ownership and sustainability of health interventions."
  },
  {
    "event": "Implementation of monitoring and evaluation processes to assess the impact of inclusive development.",
    "cause": "Need to measure the effectiveness of incorporating inclusive development into healthcare interventions.",
    "action": "Developed and implemented monitoring and evaluation frameworks focusing on inclusivity.",
    "result": "Obtained data on the impact of inclusive practices, informing future program improvements.",
    "significance": "Provided evidence to support the effectiveness of inclusive development, influencing policy and program design."
  },
  {
    "event": "USAID's support for telemedicine and mental health initiatives.",
    "cause": "Challenges in accessing healthcare services, especially in remote areas, and increasing mental health concerns.",
    "action": "Promoted telemedicine solutions and mental health programs through partnerships and technology.",
    "result": "Improved access to healthcare and mental health services for underserved populations.",
    "significance": "Addressed critical gaps in healthcare delivery and contributed to overall well-being."
  },
  {
    "event": "Advancement of innovative healthcare models and high-impact solutions through partnerships.",
    "cause": "Complex health challenges in India requiring novel approaches.",
    "action": "Tested and piloted healthcare models and innovations with public and private sector entities.",
    "result": "Development of scalable solutions to health challenges in India.",
    "significance": "Enabled replication of successful models worldwide, enhancing global health outcomes."
  }
]


# Convert the dataset into a DataFrame for easier manipulation
data_rows = []
for entry in cars_dataset:
    # Split each component into sentences (if necessary)
    for component in ['event', 'cause', 'action', 'result', 'significance']:
        text = entry[component]
        sentences = sent_tokenize(text)
        for sentence in sentences:
            data_rows.append({'sentence': sentence, 'label': component.capitalize()})

df = pd.DataFrame(data_rows)

# Map labels to integers
label_map = {'Event': 0, 'Cause': 1, 'Action': 2, 'Result': 3, 'Significance': 4}

df['label'] = df['label'].map(label_map)

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['sentence'].tolist(),
    df['label'].tolist(),
    test_size=0.1,
    random_state=42
)

# =====================================
# Step 2: Tokenization
# =====================================

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128
)

val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=128
)

# =====================================
# Step 3: Create Dataset Class
# =====================================

class CARSDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(device)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CARSDataset(train_encodings, train_labels)
val_dataset = CARSDataset(val_encodings, val_labels)

# =====================================
# Step 4: Initialize the Model
# =====================================

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_map),
    output_attentions=False,
    output_hidden_states=False
).to(device)

# =====================================
# Step 5: Training Arguments
# =====================================

training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=5,              # Total number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=50,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

# =====================================
# Step 6: Define Metrics
# =====================================

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# =====================================
# Step 7: Initialize Trainer
# =====================================

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# =====================================
# Step 8: Train the Model
# =====================================

trainer.train()

# Save the model
trainer.save_model('./trained_model')
tokenizer.save_pretrained('./trained_model')

# =====================================
# Step 9: Evaluate the Model
# =====================================

eval_results = trainer.evaluate()
print(f"\nEvaluation results:\n{eval_results}\n")

# =====================================
# Step 10: Inference on New Texts
# =====================================

def extract_cars_components(text, model, tokenizer, label_map):
    # Split the text into sentences
    sentences = sent_tokenize(text)

    # Tokenize sentences
    encodings = tokenizer(
        sentences,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='pt'
    ).to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model(**encodings)
        logits = outputs.logits
        predicted_class_ids = torch.argmax(logits, dim=1)
        predicted_labels = predicted_class_ids.cpu().numpy()

    # Map IDs to labels
    id_to_label = {v: k for k, v in label_map.items()}
    predicted_labels = [id_to_label[id] for id in predicted_labels]

    # Organize sentences by labels
    cars_components = {'Cause': [], 'Action': [], 'Result': [], 'Significance': [], 'Event': []}
    for sentence, label in zip(sentences, predicted_labels):
        if label in cars_components:
            cars_components[label].append(sentence)

    return cars_components

# Example usage:

# Load the trained model and tokenizer (if not already in memory)
# model = BertForSequenceClassification.from_pretrained('./trained_model').to(device)
# tokenizer = BertTokenizerFast.from_pretrained('./trained_model')

# Sample test text
test_text = """
USAID's partnership with local communities has led to significant improvements in healthcare access.
By recognizing the unique needs of marginalized groups, they have tailored their programs accordingly.
New clinics have been established in rural areas.
As a result, there has been a notable decrease in disease prevalence.
This approach demonstrates the impact of community-focused healthcare initiatives.
"""

# Extract CARS components
cars_components = extract_cars_components(test_text, model, tokenizer, label_map)

# Print the extracted components
for component, sentences in cars_components.items():
    print(f"\n{component}:")
    for s in sentences:
        print(f"- {s}")


/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


test code for summarization